In [1]:
import os
import ast
import shutil as sh
from pathlib import Path
import random

import numpy as np
import pandas as pd

import PIL

import torch

from tqdm.auto import tqdm

from IPython.display import Image, clear_output

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [13]:
!pip install -qr /kaggle/working/yolov5/requirements.txt

# Convert annotations to dataframe

In [14]:
import json

In [16]:
ANNOTATION_DIR = "/kaggle/input/signaturelogo-images-annotation-v2/SignatureLogo_images_annotation/ann"

In [17]:
def create_dataframe(img_name, data, df):

  for obj in data['objects']:
    label = obj['classTitle']
    bounding_box = obj['points']['exterior']
    width = bounding_box[1][0] - bounding_box[0][0]
    height = bounding_box[1][1] - bounding_box[0][1]
    final_list = []
    all_bounds = []
    for bounds in bounding_box:
      final_list.append(bounds[0])
      final_list.append(bounds[1])
    final_list = tuple(final_list)
    all_bounds.append(final_list)
    input_data = {
        "img_name":img_name,
        "class":label,
        "bounds":all_bounds,
        "width":width,
        "height":height,
    }

    tmp_df = pd.DataFrame(input_data)
    if(df.shape[0] == 0):
      df = tmp_df.copy()
    else:
      df = pd.concat([df, tmp_df], axis=0, ignore_index=True)
  return df


files_list = os.listdir(ANNOTATION_DIR)
df = pd.DataFrame()
for files in files_list:
  img_name = files.split(".json")[0]
  with open(f"{ANNOTATION_DIR}/{files}") as f:
    data = json.load(f)
  df = create_dataframe(img_name, data, df)

In [19]:
df.head()

,img_name,class,bounds,width,height
0,Detailed_Divorce_Agreement_2_pg2.jpg,Signature,"(609, 1275, 1390, 1665)",781,390
1,Detailed_Divorce_Agreement_2_pg2.jpg,Signature,"(616, 2201, 1377, 2516)",761,315
2,Detailed_Divorce_Agreement_2_pg2.jpg,Signature,"(623, 3100, 1390, 3394)",767,294
3,Detailed_Divorce_Agreement_2_pg2.jpg,Signature,"(602, 3902, 1452, 4251)",850,349
4,Detailed_Divorce_Agreement_5_pg3.jpg,Logo,"(643, 1879, 1431, 2578)",788,699


# Tile Images

In [20]:
val_df = df.iloc[45:]
val_index = val_df['img_name'].unique()
val_index

array(['Detailed_Divorce_Agreement_1_pg2.jpg',
       'Detailed_Divorce_Agreement_1_pg3.jpg'], dtype=object)

In [21]:
import os
import tqdm.notebook
TILE_WIDTH = 1200
TILE_HEIGHT = 1200
TILE_OVERLAP = 64
TRUNCATED_PERCENT = 0.3
_overwriteFiles = True

TILES_DIR = {'train': Path('train3/images'),
             'val': Path('val3/images/')}
for _, folder in TILES_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define augmentation pipeline
def get_augmentation_pipeline():
    return A.Compose([
        A.HorizontalFlip(p=0.1),
        A.RandomBrightnessContrast(p=0.2),
        A.Rotate(limit=15, p=0.1),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
        A.Resize(height=TILE_HEIGHT, width=TILE_WIDTH, always_apply=True),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Initialize augmentation pipeline
augmentation_pipeline = get_augmentation_pipeline()

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [23]:
LABELS_DIR = {'train': Path('train3/labels/'),
              'val': Path('val3/labels/')}
for _, folder in LABELS_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [24]:
IMG_DIR = "/kaggle/input/signaturelogo-images-annotation-v2/SignatureLogo_images_annotation/img"
img_list = os.listdir(IMG_DIR)

In [25]:
len(img_list)

20

In [26]:
IMAGE_WIDTH = 4250
IMAGE_HEIGHT = 5500

In [27]:
def tag_is_inside_tile(bounds, x_start, y_start, width, height, truncated_percent):
    x_min, y_min, x_max, y_max = bounds
    x_min, y_min, x_max, y_max = x_min - x_start, y_min - y_start, x_max - x_start, y_max - y_start

    if (x_min > width) or (x_max < 0.0) or (y_min > height) or (y_max < 0.0):
        return None

    x_max_trunc = min(x_max, width)
    x_min_trunc = max(x_min, 0)
    if (x_max_trunc - x_min_trunc) / (x_max - x_min) < truncated_percent:
        return None

    y_max_trunc = min(y_max, width)
    y_min_trunc = max(y_min, 0)
    if (y_max_trunc - y_min_trunc) / (y_max - y_min) < truncated_percent:
        return None

    x_center = (x_min_trunc + x_max_trunc) / 2.0 / width
    y_center = (y_min_trunc + y_max_trunc) / 2.0 / height
    x_extend = (x_max_trunc - x_min_trunc) / width
    y_extend = (y_max_trunc - y_min_trunc) / height

    return (0, x_center, y_center, x_extend, y_extend)

for img_path in tqdm.notebook.tqdm(img_list):
    pil_img = PIL.Image.open(f"{IMG_DIR}/{img_path}", mode='r')
    np_img = np.array(pil_img, dtype=np.uint8)

    img_labels = df[df["img_name"] == img_path]
    X_TILES = (IMAGE_WIDTH + TILE_WIDTH - TILE_OVERLAP - 1) // (TILE_WIDTH - TILE_OVERLAP)
    Y_TILES = (IMAGE_HEIGHT + TILE_HEIGHT - TILE_OVERLAP - 1) // (TILE_HEIGHT - TILE_OVERLAP)

    for x in range(X_TILES):
        for y in range(Y_TILES):

            x_end = min((x + 1) * TILE_WIDTH - TILE_OVERLAP * (x != 0), IMAGE_WIDTH)
            x_start = x_end - TILE_WIDTH
            y_end = min((y + 1) * TILE_HEIGHT - TILE_OVERLAP * (y != 0), IMAGE_HEIGHT)
            y_start = y_end - TILE_HEIGHT

            folder = 'val' if img_path in val_index else 'train'
            save_tile_path = TILES_DIR[folder].joinpath(img_path + "_" + str(x_start) + "_" + str(y_start) + ".jpg")
            save_label_path = LABELS_DIR[folder].joinpath(img_path + "_" + str(x_start) + "_" + str(y_start) + ".txt")

            cut_tile = np.zeros(shape=(TILE_WIDTH, TILE_HEIGHT, 3), dtype=np.uint8)
            cut_tile[0:TILE_HEIGHT, 0:TILE_WIDTH, :] = np_img[y_start:y_end, x_start:x_end, :]


            found_tags = [
                tag_is_inside_tile(bounds, x_start, y_start, TILE_WIDTH, TILE_HEIGHT, TRUNCATED_PERCENT)
                for i, bounds in enumerate(img_labels['bounds'])]
            found_tags = [el for el in found_tags if el is not None]

            if len(found_tags) > 0:
                for dup_index in range(30):  # Duplicate 10 times
                    duplicated_tile_path = TILES_DIR[folder].joinpath(
                        img_path + "_" + str(x_start) + "_" + str(y_start) + f"_dup{dup_index}.jpg"
                    )
                    duplicated_label_path = LABELS_DIR[folder].joinpath(
                        img_path + "_" + str(x_start) + "_" + str(y_start) + f"_dup{dup_index}.txt"
                    )

                    # Save duplicated image
                    duplicated_tile_img = PIL.Image.fromarray(cut_tile)
                    duplicated_tile_img.save(duplicated_tile_path)

                    # Save duplicated labels
                    with open(duplicated_label_path, 'w+') as f:
                        for tags in found_tags:
                            f.write(' '.join(str(x) for x in tags) + '\n')

            else:
              if _overwriteFiles or not os.path.isfile(save_tile_path):
                    cut_tile_img = PIL.Image.fromarray(cut_tile)
                    cut_tile_img.save(save_tile_path)
            with open(save_label_path, 'w+') as f:
                for tags in found_tags:
                    f.write(' '.join(str(x) for x in tags) + '\n')

  0%|          | 0/20 [00:00<?, ?it/s]

In [28]:
def tag_is_inside_tile(bounds, x_start, y_start, width, height, truncated_percent):
    x_min, y_min, x_max, y_max = bounds
    x_min, y_min, x_max, y_max = x_min - x_start, y_min - y_start, x_max - x_start, y_max - y_start

    if (x_min > width) or (x_max < 0.0) or (y_min > height) or (y_max < 0.0):
        return None

    x_max_trunc = min(x_max, width)
    x_min_trunc = max(x_min, 0)
    if (x_max_trunc - x_min_trunc) / (x_max - x_min) < truncated_percent:
        return None

    y_max_trunc = min(y_max, width)
    y_min_trunc = max(y_min, 0)
    if (y_max_trunc - y_min_trunc) / (y_max - y_min) < truncated_percent:
        return None

    x_center = (x_min_trunc + x_max_trunc) / 2.0 / width
    y_center = (y_min_trunc + y_max_trunc) / 2.0 / height
    x_extend = (x_max_trunc - x_min_trunc) / width
    y_extend = (y_max_trunc - y_min_trunc) / height

    return (0, x_center, y_center, x_extend, y_extend)

for img_path in tqdm.notebook.tqdm(img_list):
    pil_img = PIL.Image.open(f"{IMG_DIR}/{img_path}", mode='r')
    np_img = np.array(pil_img, dtype=np.uint8)

    img_labels = df[df["img_name"] == img_path]
    X_TILES = (IMAGE_WIDTH + TILE_WIDTH - TILE_OVERLAP - 1) // (TILE_WIDTH - TILE_OVERLAP)
    Y_TILES = (IMAGE_HEIGHT + TILE_HEIGHT - TILE_OVERLAP - 1) // (TILE_HEIGHT - TILE_OVERLAP)

    for x in range(X_TILES):
        for y in range(Y_TILES):

            x_end = min((x + 1) * TILE_WIDTH - TILE_OVERLAP * (x != 0), IMAGE_WIDTH)
            x_start = x_end - TILE_WIDTH
            y_end = min((y + 1) * TILE_HEIGHT - TILE_OVERLAP * (y != 0), IMAGE_HEIGHT)
            y_start = y_end - TILE_HEIGHT

            folder = 'val' if img_path in val_index else 'train'
            save_tile_path = TILES_DIR[folder].joinpath(img_path + "_" + str(x_start) + "_" + str(y_start) + ".jpg")
            save_label_path = LABELS_DIR[folder].joinpath(img_path + "_" + str(x_start) + "_" + str(y_start) + ".txt")

            if _overwriteFiles or not os.path.isfile(save_tile_path):
                cut_tile = np.zeros(shape=(TILE_WIDTH, TILE_HEIGHT, 3), dtype=np.uint8)
                cut_tile[0:TILE_HEIGHT, 0:TILE_WIDTH, :] = np_img[y_start:y_end, x_start:x_end, :]
                cut_tile_img = PIL.Image.fromarray(cut_tile)
                cut_tile_img.save(save_tile_path)

            found_tags = [
                tag_is_inside_tile(bounds, x_start, y_start, TILE_WIDTH, TILE_HEIGHT, TRUNCATED_PERCENT)
                for i, bounds in enumerate(img_labels['bounds'])]
            found_tags = [el for el in found_tags if el is not None]

            # save labels
            with open(save_label_path, 'w+') as f:
                for tags in found_tags:
                    f.write(' '.join(str(x) for x in tags) + '\n')

  0%|          | 0/20 [00:00<?, ?it/s]

In [34]:
# Yaml file for YOLO

CONFIG = """
# train and val datasets (image directory or *.txt file with image paths)
train: /kaggle/working/train3/
val: /kaggle/working/val3/

# number of classes
nc: 2

# class names
names: ['Signature','Logo']
"""

with open("dataset.yaml", "w") as f:
    f.write(CONFIG)

In [35]:
!python /kaggle/working/yolov5/train.py --cfg yolov5s.yaml --imgsz 1200 --batch-size 16 --epochs 10 --data dataset.yaml --weights yolov5s.pt

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-21 10:01:15.342881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-21 10:01:15.364471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-21 10:01:15.371830: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [59]:
!python /kaggle/working/yolov5/detect.py --source /kaggle/input/signaturelogo-images-annotation-v2/SignatureLogo_images_annotation/img --img-size 4250 5500 --weights /kaggle/working/yolov5/runs/train/exp3/weights/best.pt --conf 0.5 --save-txt

detect: weights=['/kaggle/working/yolov5/runs/train/exp3/weights/best.pt'], source=/kaggle/input/signaturelogo-images-annotation-v2/SignatureLogo_images_annotation/img, data=yolov5/data/coco128.yaml, imgsz=[4250, 5500], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ --img-size [4250, 5500] must be multiple of max stride 32, updating to [4256, 5504]
image 1/20 /kaggle/input/signaturelogo-images-annotation-v2/SignatureLogo_images_annotation/img/Detailed_Divorce_A

In [39]:
# import glob
# from IPython.display import Image, display

# for image_path in glob.glob('yolov5/runs/detect/exp/*.jpg'):
#       display(Image(filename=image_path, width=1024))
#       print("\n")

# Gliner

In [40]:
!pip install gliner
!pip install PyMuPDF
!pip install fpdf
!pip install pdf2image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 80.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=0aa932764e853938e05d4b0c729ad1209d358e31756933cfbd6f025187d3c613
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [41]:
import fitz  # PyMuPDF
from gliner import GLiNER

# --- Initialize GLiNER ---
#model = GLiNER.from_pretrained("urchade/gliner_large-v2.1")
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")
labels = ["Person", "Money", "Date", "Tenure", "Country", "Street_Address", "State", "City"]

def replace_text_with_labels(input_path, output_path):
    doc = fitz.open(input_path)
    fontname = "Helvetica"

    for page_num in range(len(doc)):
        page = doc[page_num]
        full_text = page.get_text()

        # Predict entities and filter overlaps
        entities = model.predict_entities(full_text, labels=labels, threshold=0.5)
        entities = resolve_overlaps_and_errors(entities)  # <-- New overlap resolution

        # Sort entities by position (reverse order to avoid coordinate shifts)
        entities.sort(key=lambda x: x["start"], reverse=True)

        # Replace entities in the PDF
        for entity in entities:
            original_text = entity["text"]
            label = entity["label"]

            # Find all occurrences of the entity on the page
            text_instances = page.search_for(original_text)

            for inst in text_instances:
                # Get original text properties
                block = find_text_block(page.get_text("dict"), inst)
                if not block:
                    continue

                # Replace text with label using standard font
                page.add_redact_annot(
                    inst,
                    text=f"[{label}]",
                    fontsize=block["size"],
                    fontname=fontname,
                    text_color=block["color"],
                    fill=(1, 1, 1)
                )

        # Apply all redactions on the page
        page.apply_redactions(images=fitz.PDF_REDACT_IMAGE_NONE)

    doc.save(output_path, garbage=4, deflate=True, incremental=False)
    doc.close()
    print(f"✅ Modified PDF saved to: {output_path}")

def resolve_overlaps_and_errors(entities):
    """Resolve overlaps AND filter incorrect labels like 'marriage' → [State]"""
    priority_labels = ["Street_Address", "State", "City", "Country"]
    resolved = []

    # Sort by length (longest first) to prioritize broader matches
    entities.sort(key=lambda x: x["end"] - x["start"], reverse=True)

    for entity in entities:
        text = entity["text"].lower()  # Case-insensitive check

        # --- Custom Rule to Fix "marriage" → [State] ---
        if entity["label"] == "State" and "Address" in text:
            continue  # Skip this entity

        # --- Custom Rule to Fix "ia" → [State] ---
        if entity["label"] == "State" and text == "ia":
            continue  # Skip standalone "ia" (if not part of a valid context)

        # --- Standard Overlap Resolution ---
        is_contained = False
        for resolved_entity in resolved:
            if (entity["start"] >= resolved_entity["start"] and
                entity["end"] <= resolved_entity["end"]):
                is_contained = True
                break

        if not is_contained:
            resolved.append(entity)

    return resolved

def find_text_block(text_dict, rect):
    """Find text block containing the given rectangle"""
    for block in text_dict["blocks"]:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                span_rect = fitz.Rect(span["bbox"])
                if span_rect.intersects(rect):
                    return span
    return None

# --- Usage ---
input_pdf = "/kaggle/input/divorce-pdf/Detailed_Divorce_Agreement_8.pdf"
output_pdf = "Modified_Divorce_Agreement_8.pdf"
replace_text_with_labels(input_pdf, output_pdf)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 464 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 449 has been truncat

✅ Modified PDF saved to: Modified_Divorce_Agreement_8.pdf


In [42]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 126 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.6 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.6 [5,184 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.6 [1,071 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 1,4

In [49]:
os.makedirs("images_gliner", exist_ok=True)

In [50]:
from pdf2image import convert_from_path
from PIL import Image
pdf_list = ['/kaggle/working/Modified_Divorce_Agreement_8.pdf']
for pdfs in pdf_list:
    file_name_image = pdfs.split(r"/")[-1].split(".")[0]
    pages = convert_from_path(pdfs, 500)

    for count, page in enumerate(pages):

        page.save(rf'images_gliner/{file_name_image}_pg{count}.jpg', 'JPEG')

In [60]:
!python /kaggle/working/yolov5/detect.py --source /kaggle/working/images_gliner --img-size 4250 5500 --weights /kaggle/working/yolov5/runs/train/exp3/weights/best.pt --conf 0.5 --save-txt

detect: weights=['/kaggle/working/yolov5/runs/train/exp3/weights/best.pt'], source=/kaggle/working/images_gliner, data=yolov5/data/coco128.yaml, imgsz=[4250, 5500], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ --img-size [4250, 5500] must be multiple of max stride 32, updating to [4256, 5504]
image 1/4 /kaggle/working/images_gliner/Modified_Divorce_Agreement_8_pg0.jpg: 4256x3296 (no detections), 187.5ms
image 2/4 /kaggle/working/images_gliner/Modified_Divorce

In [67]:
import cv2
import os

# Paths
source_dir = "/kaggle/working/yolov5/runs/detect/exp8"  # Directory containing detected images
labels_dir = "/kaggle/working/yolov5/runs/detect/exp8/labels"  # Directory containing detection .txt files
output_dir = "/kaggle/working/yolov5/runs/detect/exp8_masked"  # Directory to save modified images


os.makedirs(output_dir, exist_ok=True)

for image_name in os.listdir(source_dir):
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Skip non-image files
        continue
    
    image_path = os.path.join(source_dir, image_name)
    label_path = os.path.join(labels_dir, os.path.splitext(image_name)[0] + ".txt")

    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()
        
        # Draw black rectangles for each detection
        for line in lines:
            parts = line.strip().split()
            
       
            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:5])
            x1 = int((x_center - bbox_width / 2) * w)
            y1 = int((y_center - bbox_height / 2) * h)
            x2 = int((x_center + bbox_width / 2) * w)
            y2 = int((y_center + bbox_height / 2) * h)
            
       
            label_height_extension = int(bbox_height * 0.5)  
            y1_extended = max(0, y1 - label_height_extension)  
            
            
            cv2.rectangle(img, (x1, y1_extended), (x2, y2), (0, 0, 0), thickness=-1)  # thickness=-1 fills the rectangle


    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, img)

print("Black masks applied and modified images saved successfully!")

Black masks applied and modified images saved successfully!


In [68]:
from PIL import Image

image_paths = [f"/kaggle/working/yolov5/runs/detect/exp8_masked/Modified_Divorce_Agreement_8_pg{i}.jpg" for i in range(4)]  # Adjust the range based on the number of images


first_image = Image.open(image_paths[0])

other_images = [Image.open(img_path).convert("RGB") for img_path in image_paths[1:]]


output_pdf_path = "Detailed_Divorce_Agreement_8_redacted.pdf"
first_image.convert("RGB").save(output_pdf_path, save_all=True, append_images=other_images)

print(f"Combined PDF saved at: {output_pdf_path}")

Combined PDF saved at: Detailed_Divorce_Agreement_8_redacted.pdf
